# Evaluation of LLM Agent against Ground Truth Labels for Spatial Clustering Methods
## Introduction 
This research project seeks to answer the following question: "Is it possible to create an agent that can understand and summarise the literature of spatial clustering methods employed in the field of spatial transcriptomics?" I plan to answer this by creating my own LLM agent and comparing it against other existing LLM models. I improved the agent by employing RAG (Retrieval Augmented Generation) and specialised web search engine Tavily as tools for the Agent. Additional system prompts also were used to help guide the Agent for the nature of its responses. Evaluation of the agent was performed by comparing a large spreadsheet of various metrics and categories for each spatial clustering method generated from human experts against the results generated by the agent. Ablation for each tool has been performed to analyse further impacts of each tool added, and finally a comparison of the agent against the end-user application of ChatGPT was performed.

## Method
The LLM Agent was developed using the Langchain framework, which allows for flexible abstractions during the development process. Langchain allows for easy integration of new features such as the use of embedding models for RAG and an AgentExecutor class for a useful interface for helping the agent choose a relevant tool when coming up with a response. 

### Model Selection
Google Gemini Flash was chosen as it was free to use for a large number of tokens for educational purposes. 

### System Prompt
The system was provided with a prompt that informs it that it is an assistant capable of answering various topics, from simple topics to deep discussions. It is also told to act as a spatial transcriptomics expert, and that answers should be kept technical, but concise. 

Additional system prompts include the definitions of various categories for each spatial algorithm as determined by human experts which can be seen [here](https://docs.google.com/spreadsheets/d/1P1-Nw0i_MpLoE8he1H7ZT-acYd4jOgDPrKZBxR-L6dw/edit?gid=0#gid=0). This ranged from things such as clustering method (Bayesian, Graph-based, Autoencoder, Centroid, Hierarchical-based), Scalability, Assumptions, Input data, programming language and metrics + simulations employed in the article. See the `prompts.py` file for more information regarding the specific details.

### Information Review Sheet
The spatial information review sheet had its categories filled in by 2 domain experts. On top of discrete categories, various columns of the sheet also contained details of the experts' thoughts of the algorithm on things such as realism, additional datasets, and comments.

### RAG Process
The RAG process begins by taking in a scientific article in the form of an input pdf file by loading it using the PyPDFLoader package. It is then broken down into several chunks using a RecursiveTextCharacterSplitter to improve proccessing efficiency, this can then be converted to vectors via an embedding model. The vectors are then stored in a Chroma vector database, which can be fetched from the LLM via a document retriever. A document retriever attempts to find the most relavant information in relation to a user query (In this case, the user's question). To improve the RAG process, an ensemble retriever was used consisting of the following: The first retriever uses a vector similarity search from the Chroma vector database by looking at vectors that were closer to the query in a high dimensional space. The second is a BM25 retriever which ranks the best match 25 chunks based on relevance to the given query.

### Tavily Search
Tavily Search API is a web search engine API that can be used by an LLM during the processing stage, it searches the web for additional relevant webpages by using the context of the user query. This helps fill in gaps and allows the Agent to infer certain details when the information of the article is insufficient for answering its questions. (I.e. Accessing the source code of the algorithm via public GitHub repository)


## Evaluation Method
With all this in mind, let us discuss the details of the evaluation. Five different methods were evaluated:

1. The Agent with all tools available.
2. The Agent without Tavily Search.
3. The Agent without document retrievers.
4. The Agent with no tools available
5. ChatGPT's end-user interface.

In each of these methods, a scientific article of a spatial clustering method was provided, along with a user prommpt that tells the agent to categorise the spatial clustering method all the fields as described in the **System** Prompt section. With the following structure:

<Category>,<Answer>

Each result was stored in the form of a csv file under the file structure <method_name>/<article_name>.csv 

This process was repeated until all 5 methods contained all csvs.

# Analysis
With all this in mind, we can now begin the coding part.

## Data Loading
Dataframes were created by accessing each directory and reading each individual csv file. The csvs were transposed and added into a dataframe one row at a time. Once all csvs were retrieved and appended, an additional column specifying the LLM method was added to the dataframe.

For the spatial clustering information sheet- the Google Sheets was directly read, specifying ranges with discrete values. And since currently the sheet is still under review, duplicate entries were removed from the dataset.

## Comparisons 
To obtain the comparison between information sheet and agent output, column names were matched so that the dataframes could be joined, and filtered by method.

Correctness of the agent was measured by counting the number of matching entries from the ground truth and agent.

In [1]:
import pandas as pd 
import os
from matplotlib import pyplot as plt
import numpy as np

def clean_file(filename):
    '''A function to clean up the output because Gemini is too dumb to answer in the specified output.
    folder: str of filename name
    returns: None'''
    with open(filename,'r') as file:
        lines = file.readlines()
    
    cleaned_lines = [line.strip("```") for line in lines if '```' not in line]
    with open(filename,'w') as file:
        file.write(''.join(cleaned_lines))

def load_sheet(folder, sim_no):
    '''Given a folder, load all csv files for each clustering methods
    folder: str of folder name
    sim_no: int of simulation number
    returns: A dataframe containing all LLM predictions for clustering categories'''
    clean_file(f"sim{sim_no}/{folder}/banksy.csv")
    # Grab the first 85 columns, have to do this beceause somehow Gemini messed up the one job it had.
    df = pd.read_csv(f"sim{sim_no}/{folder}/banksy.csv", index_col= 0, skipinitialspace= True).transpose()
    df = df.iloc[:,:85]
    df.index = df.index.str.strip()

    for sheet in os.listdir(f"sim{sim_no}/{folder}")[1:]:
        try: 
            if sheet.endswith(".csv"):
                clean_file(f"sim{sim_no}/{folder}/{sheet}")
                tmp_df = pd.read_csv(f"sim{sim_no}/{folder}/{sheet}", index_col= 0, skipinitialspace= True).transpose()
                # If there isn't the right columns after all this, you are disqualified!
                if tmp_df.shape[1] < 85:
                    tmp_df = pd.DataFrame([["None"] * len(df.columns)], columns = df.columns,
                                       index = [sheet[:-4].upper()]
                                       )
                    tmp_df.index = tmp_df.index.str.strip()
                    df = pd.concat([df,tmp_df], axis = 0)
                    continue
                tmp_df = tmp_df.iloc[:,:85]
                tmp_df.index = tmp_df.index.str.strip()
                tmp_df.columns = df.columns 
                df = pd.concat([df,tmp_df], axis = 0)
        except Exception as e:
            print(e)
            print(f"Error in {folder}")
            print("Adding in:")
            print(tmp_df)
            print("Current")
            print(df)
            break
    

    if df.columns[-1] == "```":
        df = df.iloc[:,:-1]
    df['Source'] = folder
    return df

def load_google_sheet():
    '''Loads the spatial clustering review information sheet from google Sheets
    returns: A dataframe containing all labels'''
    gsheetid = "1P1-Nw0i_MpLoE8he1H7ZT-acYd4jOgDPrKZBxR-L6dw"
    sheet_name = "Methods" 
    online_sheet = f"https://docs.google.com/spreadsheets/d/{gsheetid}/gviz/tq?tqx=out:csv&sheet={sheet_name}&range=A2:A,I2:Y,AA2:AE,AH2:AW,AX2:BI,BL2:BQ,BS2:BU,BW2:CO,CQ2:CR,CT2:CX"
    info = pd.read_csv(online_sheet, index_col = 0)
    info = info.dropna(subset = ["Parameter testing"])
    info['Source'] = "Truth"

    return info

def filter_and_compare(llm_df, truth):
    '''Joins a method with a specified LLM category and the ground truth dataframe together and 
    performs a comparison, by calculating the number of matching terms.
    
    llm_df (df): Dataframe of LLM
    truth (df): Ground Truth dataframe
    method (str): Method to filter on 
    returns final: A dataframe containing the correctness of each algorithm'''
    methods = ['agent_out', 'gpt_out', 'search_out', 'pdf_out']
    dfs = []
    for method in methods:
        agent_filter = llm_df[llm_df['Source'] == method]
        agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
        truth['Algorithm'] = truth['Algorithm'].str.upper()

        truth_filter = truth[truth['Algorithm'].isin(agent_filter['Algorithm'])]
        agent_filter = agent_filter[agent_filter['Algorithm'].isin(truth['Algorithm'])]

        agent_filter = agent_filter.sort_values(by= 'Algorithm').reset_index(drop = True)
        truth_filter = truth_filter.sort_values(by= 'Algorithm').reset_index(drop = True)

        compare = agent_filter.eq(truth_filter)
        compare['Algorithm'] = agent_filter['Algorithm']
        compare['Correct_Terms'] = compare.drop(columns='Algorithm').sum(axis = 1)
        compare['Correct_Percent'] = 100* compare['Correct_Terms'] / compare.shape[1]
        compare['Method'] = method 
        dfs.append(compare)

    final = pd.concat(dfs)
    return final

def evaluate(sim_no):
    '''Whole evaluation for entire pipeline for simulation i'''
    agent = load_sheet("agent_out", sim_no)
    gpt = load_sheet("gpt_out", sim_no)
    pdfs = load_sheet("pdf_out", sim_no)
    search = load_sheet("search_out", sim_no)
    truth = load_google_sheet()
    truth = truth[~truth.index.duplicated(keep='first')]
    truth.reset_index(inplace= True)

    llm_df = pd.concat([agent, gpt, search, pdfs])
    llm_df.reset_index(inplace = True)
    llm_df.rename(columns = {"index":"Algorithm"}, inplace = True)
    truth.columns = llm_df.columns
    llm_df.columns.name = None
    print(filter_and_compare(llm_df, truth))
    return filter_and_compare(llm_df, truth)

df_list = []
for i in range(1,6):
    df_list.append(evaluate(i))

C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

    Algorithm  Package  Bioconductor  CRAN  Vignette  \
0        BASS     True          True  True      True   
1      DEEPST     True          True  True     False   
2     GRAPHST     True          True  True     False   
3        SEDR     True          True  True     False   
4   SPACEFLOW     True          True  True     False   
5      SPAGCN     True          True  True     False   
6  SPATIALPCA     True          True  True     False   
7     STAGATE     True          True  True     False   
0        BASS    False          True  True     False   
1      DEEPST    False          True  True      True   
2     GRAPHST    False          True  True      True   
3        SEDR    False          True  True      True   
4   SPACEFLOW    False          True  True      True   
5      SPAGCN    False          True  True      True   
6  SPATIALPCA    False          True  True      True   
7     STAGATE    False          True  True      True   
0        BASS     True          True  True      

C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

      Algorithm  Package  Bioconductor   CRAN  Vignette  \
0          BASS     True          True   True      True   
1        DEEPST     True          True   True      True   
2      GRAPHPCA     True          True   True      True   
3       GRAPHST     True          True   True      True   
4    MULTILAYER    False          True   True      True   
5      NICHEPCA     True          True   True      True   
6          SEDR     True          True   True      True   
7     SPACEFLOW     True          True   True      True   
8        SPAGCN     True          True   True      True   
9      SPARTACO     True          True   True      True   
10  SPATIALANNO     True          True   True      True   
11   SPATIALPCA     True          True   True      True   
12      STAGATE     True          True   True      True   
13       STKEEP     True          True   True      True   
14        STSME     True          True   True      True   
0          BASS     True          True   True      True 

C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

      Algorithm  Package  Bioconductor   CRAN  Vignette  \
0          BASS     True          True   True      True   
1        DEEPST     True          True   True      True   
2      GRAPHPCA     True          True   True      True   
3       GRAPHST     True          True   True      True   
4    MULTILAYER    False          True   True      True   
5      NICHEPCA     True          True   True      True   
6          SEDR     True          True   True      True   
7     SPACEFLOW     True          True   True      True   
8        SPAGCN     True          True   True      True   
9      SPARTACO     True          True   True      True   
10  SPATIALANNO     True          True   True      True   
11   SPATIALPCA     True          True   True      True   
12      STAGATE     True          True   True      True   
13       STKEEP     True          True   True      True   
14        STSME     True          True   True      True   
0          BASS    False          True   True      True 

C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

      Algorithm  Package  Bioconductor   CRAN  Vignette  \
0          BASS     True          True   True      True   
1        DEEPST     True          True   True      True   
2      GRAPHPCA     True          True   True      True   
3       GRAPHST     True          True   True      True   
4    MULTILAYER    False          True   True      True   
5      NICHEPCA     True          True   True      True   
6          SEDR     True          True   True      True   
7     SPACEFLOW     True          True   True      True   
8        SPAGCN     True          True   True      True   
9      SPARTACO     True          True   True      True   
10  SPATIALANNO     True          True   True      True   
11   SPATIALPCA     True          True   True      True   
12      STAGATE     True          True   True      True   
13       STKEEP     True          True   True      True   
14        STSME     True          True   True      True   
0          BASS    False          True   True     False 

C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_filter['Algorithm'] =  agent_filter['Algorithm'].str.upper()
C:\Users\chuah\AppData\Local\Temp\ipykernel_1428\1945817202.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data